In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

**Data Preprocessing**

In [3]:
import os
import numpy as np
import cv2
def load_images_from_folder(folder_path, target_size=(64, 64)):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
        img = cv2.resize(img, target_size)
        img = img.astype(np.float32) / 255.0  
        images.append(img)
    return np.array(images)
    
folder_path = fr"C:\Users\surri\PycharmProjects\Python Project\Data 2"
high_res_images = load_images_from_folder(folder_path)

In [4]:
def blur_images(images, kernel_size=(5, 5)):
    blurred_images = []
    for img in images:
        blurred_img = cv2.GaussianBlur(img, kernel_size, 0)
        blurred_images.append(blurred_img)
    return np.array(blurred_images)
blurred_images = blur_images(high_res_images)


**Building the Generator**

In [5]:
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,LeakyReLU,UpSampling2D,Flatten,Reshape,Input,BatchNormalization

In [6]:
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, Add, BatchNormalization, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def build_generator():
    input_lr = Input(shape=(64, 64, 3))
    
    # Encoder part
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input_lr)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    # Bottleneck block
    for _ in range(6):
        residual = x
        x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
        x = BatchNormalization()(x)
        x = Add()([x, residual])
        x = LeakyReLU(alpha=0.2)(x)

    # Decoder part
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)
    
    output_hr = Conv2D(3, kernel_size=3, activation='sigmoid', padding='same')(x)
    generator = Model(input_lr, output_hr, name='generator') 
    return generator


In [7]:
generator = build_generator()

In [8]:
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 64, 64, 64)   1792        ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 64, 64, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 32, 128)  73856       ['leaky_re_lu[0][0]']            
                                                                                          

**Build Discriminator**

In [9]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

In [10]:
def build_discriminator():
    model = Sequential()
    
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same", input_shape=(64, 64, 3)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(256, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    return model
    
discriminator = build_discriminator()
discriminator.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_44 (Conv2D)          (None, 32, 32, 256)       12544     
                                                                 
 leaky_re_lu_42 (LeakyReLU)  (None, 32, 32, 256)       0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 256)       0         
                                                                 
 conv2d_45 (Conv2D)          (None, 16, 16, 256)       1048832   
                                                                 
 batch_normalization_40 (Bat  (None, 16, 16, 256)      1024      
 chNormalization)                                                
                                                                 
 leaky_re_lu_43 (LeakyReLU)  (None, 16, 16, 256)       0         
                                                        

**Create Custom Model**

In [11]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [12]:
g_opt = Adam(0.0001)
d_opt = Adam(0.00001)
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [14]:
class Super_Res(Model):

    def __init__(self, generator, discriminator, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.generator = generator
        self.discriminator = discriminator
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, data):
        real_images, blurred_images = data
        fake_images = self.generator(blurred_images, training=False)

        with tf.GradientTape() as d_tape:
            yhat_real = self.discriminator(real_images, training=True)
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            noise_real = 0.1*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.1*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)

        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))

        with tf.GradientTape() as g_tape:
            gen_images = self.generator(blurred_images, training=True)
            predicted_labels = self.discriminator(gen_images, training=False)
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels)

        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))

        return {"d_loss": total_d_loss, "g_loss": total_g_loss}


**Training the Model**

In [15]:
super_res_model = Super_Res(generator, discriminator)

In [16]:
super_res_model.compile(g_opt, d_opt, g_loss, d_loss)

In [17]:
batch_size = 31
num_epochs = 20

num_batches = len(high_res_images) // batch_size
high_res_batches = np.array_split(high_res_images[:num_batches * batch_size], num_batches)
blurred_batches = np.array_split(blurred_images[:num_batches * batch_size], num_batches)
for epoch in range(num_epochs):
        for high_res_batch, blurred_batch in zip(high_res_batches, blurred_batches):
            loss = super_res_model.train_step((high_res_batch, blurred_batch))
        print(f"Epoch {epoch + 1}, d_loss: {loss['d_loss']}, g_loss: {loss['g_loss']}")

Epoch 1, d_loss: 0.9130052328109741, g_loss: 0.04972480982542038
Epoch 2, d_loss: 0.7131525278091431, g_loss: 0.0026331671979278326
Epoch 3, d_loss: 0.4677678644657135, g_loss: 0.0006547514931298792
Epoch 4, d_loss: 0.46129879355430603, g_loss: 0.0007101114606484771
Epoch 5, d_loss: 0.49202480912208557, g_loss: 0.0010312676895409822
Epoch 6, d_loss: 0.6264033317565918, g_loss: 0.0010316673433408141
Epoch 7, d_loss: 0.6871498823165894, g_loss: 0.0011519469553604722
Epoch 8, d_loss: 0.887736439704895, g_loss: 0.00137179100420326
Epoch 9, d_loss: 0.7814577221870422, g_loss: 0.0013565723784267902
Epoch 10, d_loss: 0.6798390746116638, g_loss: 0.001965646632015705
Epoch 11, d_loss: 0.6022716164588928, g_loss: 0.0029916740022599697
Epoch 12, d_loss: 0.612019956111908, g_loss: 0.002687879139557481
Epoch 13, d_loss: 0.6277465224266052, g_loss: 0.001964757451787591
Epoch 14, d_loss: 0.4983743131160736, g_loss: 0.0018301131203770638
Epoch 15, d_loss: 0.577125072479248, g_loss: 0.00234160968102514